In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import openai
import os
from dotenv import load_dotenv
import json

In [29]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


RECIPE = {
    "title": "Creamy Mac & Cheese",
    "ingredients": [
        "2 cups elbow macaroni",
        "2 cups shredded cheddar cheese",
        "1 ½ cups milk",
        "2 tbsp butter",
        "2 tbsp all-purpose flour",
        "1 tsp salt",
        "½ tsp black pepper",
        "½ tsp garlic powder"
    ],
    "steps": [
        "Boil water in a pot, add a pinch of salt.",
        "Cook macaroni until it's firm but tender when bitten, then drain and set aside.",
        "In a saucepan, melt butter over medium heat.",
        "Add flour and whisk for 1-2 minutes until it turns slightly golden.",
        "Slowly pour in the milk while whisking to avoid lumps.",
        "Add salt, pepper, and garlic powder.",
        "Mix in optional ingredients like spinach, grilled chicken, or yogurt for creaminess.",
        "Add cooked pasta to the cheese sauce and mix well."
    ]
}

user_state = {
    "current_step": 0,
    "mistakes": []
}

user_history = {}

In [23]:
def answer_question(user_id, user_message):
    """Answer the user's question based on the recipe or general knowledge while keeping conversation history."""
    if user_id not in user_history:
        user_history[user_id] = [
            {"role": "system", "content": f"""
                You are a professional chef specializing in answering recipe-related questions.
                Here is the recipe data:
                {json.dumps(RECIPE, indent=2)}
                
                User's current step: {RECIPE["steps"][user_state['current_step']]}

                Answer questions based only on this recipe and general cooking knowledge. Shorten answers to at most 2 sentences.
            """}
        ]

    # Add user message to history
    user_history[user_id].append({"role": "user", "content": user_message})

    # Call OpenAI API with conversation history
    response = openai.ChatCompletion.create(
        model="gpt-4",
        api_key=OPENAI_API_KEY,
        messages=user_history[user_id]
    )

    bot_response = response["choices"][0]["message"]["content"].strip()

    # Add AI response to history
    user_history[user_id].append({"role": "assistant", "content": bot_response})

    # Limit message history to last 10 messages to save tokens
    user_history[user_id] = user_history[user_id][-10:]

    return bot_response

In [34]:
def change_recipe(preference):
    """Change recipe based on user's preference"""
    system_msg = f"""You are a helpful recipe advisor who can modify recipe based on user's preferences.
                Here is the original recipe data:
                {json.dumps(RECIPE, indent=2)}
                
                You must always respond with the following JSON object: 
                {{
                    "message": (Example: Since you are vegan, I have modified the recipe to be vegan friendly, such as changing ... summary it in 1 paragraph),
                    "modified_recipe": {{
                        "title": modified title (For example: Vegan Mac & Cheese)
                        "ingredients": list of modified ingredients (ONLY CHANGE THE NECESSARY ONES),
                        "steps": list of modified steps (ONLY CHANGE THE NECESSARY ONES)
                    }}
                }}
                You must return this JSON object directly, without any apologies, explanations, or additional text.
                """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        api_key=OPENAI_API_KEY,
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": preference}
        ]
    )

    return response["choices"][0]["message"]["content"].strip().replace("\u00bd", "½")

In [35]:
print(change_recipe("i don't want garlic"))

{
  "message": "I've removed the garlic in the recipe to accommodate your preference.",
  "modified_recipe": {
    "title": "Creamy Mac & Cheese (No Garlic)",
    "ingredients": [
      "2 cups elbow macaroni",
      "2 cups shredded cheddar cheese",
      "1 ½ cups milk",
      "2 tbsp butter",
      "2 tbsp all-purpose flour",
      "1 tsp salt",
      "½ tsp black pepper"
    ],
    "steps": [
      "Boil water in a pot, add a pinch of salt.",
      "Cook macaroni until it's firm but tender when bitten, then drain and set aside.",
      "In a saucepan, melt butter over medium heat.",
      "Add flour and whisk for 1-2 minutes until it turns slightly golden.",
      "Slowly pour in the milk while whisking to avoid lumps.",
      "Add salt and pepper.",
      "Mix in optional ingredients like spinach, grilled chicken, or yogurt for creaminess.",
      "Add cooked pasta to the cheese sauce and mix well."
    ]
  }
}
